# Core Model Guide: SCSRunoffModel

This notebook provides a detailed guide to the `SCSRunoffModel`, which implements the widely-used **SCS Curve Number (CN)** method for estimating direct runoff from a rainfall event.

## 1. Theoretical Background

The SCS-CN method is an empirical model developed by the USDA Natural Resources Conservation Service (NRCS). It's based on the following relationship:

```
Q = (P - Ia)² / (P - Ia + S)
```

Where:
- `Q` is the direct runoff depth.
- `P` is the total rainfall depth.
- `Ia` is the **initial abstraction**, representing all losses before runoff begins (e.g., interception by vegetation, surface storage, infiltration). It is empirically related to `S` by `Ia = 0.2 * S`.
- `S` is the **potential maximum retention** after runoff begins. It is related to the Curve Number (CN) by the formula:

```
S = (1000 / CN) - 10
```

The **Curve Number (CN)** is a dimensionless parameter ranging from 30 to 100, which represents the runoff potential of an area based on its soil type, land use, and antecedent moisture conditions.

## 2. API and Parameters

The `SCSRunoffModel` is a very simple class. Its `calculate_runoff` method takes the following arguments:

- `rainfall` (float): The rainfall depth for the time step.
- `sub_basin_params` (dict): A dictionary containing the parameters for the basin. The model expects a `"CN"` key in this dictionary.
- `dt` (float): The time step duration (not directly used in the formula but part of the standard interface).

In [ ]:
import sys
import os
import matplotlib.pyplot as plt
import numpy as np

# Add the project root to the path
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

from water_system_sdk.src.chs_sdk.modules.modeling.hydrology.runoff_models import SCSRunoffModel

## 3. Code Example: Rainfall-Runoff Relationship

Let's instantiate the model and see how the runoff changes for different rainfall amounts for a given Curve Number.

In [ ]:
# 1. Model Initialization
scs_model = SCSRunoffModel()

# 2. Define Parameters
cn = 85 # A moderately high curve number
params = {"CN": cn}
dt = 1.0 # Not used, but required by the interface

# 3. Run for a range of rainfall values
rainfall_values = np.linspace(0, 10, 100) # Test rainfall from 0 to 10
runoff_values = []

for p in rainfall_values:
    runoff = scs_model.calculate_runoff(rainfall=p, sub_basin_params=params, dt=dt)
    runoff_values.append(runoff)

print(f"Runoff calculation for CN={cn} complete.")

## 4. Visualization

Plotting rainfall vs. runoff shows the characteristic nonlinear relationship of the SCS-CN method.

In [ ]:
# Let's also calculate S and Ia for context
S = (1000 / cn) - 10
Ia = 0.2 * S

plt.figure(figsize=(10, 6))
plt.plot(rainfall_values, runoff_values, label=f'Runoff (CN={cn})', color='blue')
plt.axvline(x=Ia, color='r', linestyle='--', label=f'Initial Abstraction (Ia) = {Ia:.2f}')
plt.title('SCS-CN Rainfall-Runoff Relationship')
plt.xlabel('Total Rainfall (P)')
plt.ylabel('Direct Runoff (Q)')
plt.legend()
plt.grid(True)
plt.show()

The plot clearly shows that runoff is zero until the total rainfall `P` exceeds the initial abstraction `Ia`. After that threshold, the runoff begins and increases nonlinearly with increasing rainfall, eventually approaching a 1:1 relationship for very high rainfall amounts.